In [ ]:
import json
from itertools import combinations
import numpy as np
from matplotlib import pyplot as plt
from skimage.draw import polygon
from skimage.morphology import skeletonize, skeletonize_3d, medial_axis
from skimage.measure import regionprops, label

## Methods definition

In [ ]:
def extract_name(feature):
    if 'name' in feature['properties']:
        return feature['properties']['name']
    elif 'wikipedia' in feature['properties']:
        return feature['properties']['wikipedia']
    else:
        return None
    

In [ ]:
def extract_coordinates(feature):
    x_coordinates = []
    y_coordinates = []
    for point in feature['geometry']['coordinates'][0]:
        x_coordinates.append(point[0])
        y_coordinates.append(point[1])
                
    return x_coordinates, y_coordinates

In [ ]:
def binary_image(im_size, x_coordinates, y_coordinates):
    x_coordinates = np.array(x_coordinates)
    y_coordinates = np.array(y_coordinates)

    x_coordinates -= min(x_coordinates)
    y_coordinates -= min(y_coordinates)


    x_max = max(x_coordinates)
    y_max = max(y_coordinates)

    if x_max >= y_max:
        ratio = im_size/x_max

    else:
        ratio = im_size/y_max

    x_coordinates *= ratio
    y_coordinates *= ratio

    img = np.zeros((im_size, im_size), dtype=np.uint8)

    x, y = polygon(x_coordinates, y_coordinates)
    img[x, y] = 1
    
    return img

## Data loading

In [ ]:
im_size = 50


In [ ]:
with open('geojson/lacs_chablais.geojson') as f:
    lacs_chablais = json.load(f)

#### Conversion to format accepted by Leonardo's code

In [ ]:
lakes_leo = []

for feature in lacs_chablais['features']:
    if feature['geometry']['type'] == 'Polygon':
        lake = {
            'id': feature['properties']['@id'].replace('/', '_'),
            'vertices': feature['geometry']['coordinates'][0],
            'name': extract_name(feature),
        }

        lakes_leo.append(lake)

In [ ]:
with open('json/lakes_leo_suisse.json', 'w') as f:
    
    f.write(json.dumps(lakes_leo))

#### TODO: Manage multipolygon

In [ ]:
lakes = []
for feature in lacs_chablais['features']:
    if feature['geometry']['type'] == 'Polygon':
        name = extract_name(feature)
        coordinates = extract_coordinates(feature)
        bin_image = binary_image(im_size, *coordinates)
        lakes.append((name, bin_image, coordinates))

In [ ]:
def logical_distance(im1, im2):
    im_xor = np.logical_xor(im1,im2)
    #plt.imshow(im_xor)
    #plt.show()
    return np.sum(im_xor)
    

In [ ]:
im1 = lakes[0][1]
im2 = lakes[1][1]

total_sum = logical_distance(im1, im2)

In [ ]:
total_sum

In [ ]:
distances = []
for lake1, lake2 in combinations(lakes,2):
    distances.append((logical_distance(lake1[1], lake2[1]), lake1, lake2))
    

In [ ]:
distances = sorted(distances, key=lambda d: d[0])

In [ ]:
#distances[0]

In [ ]:
distance = distances[16]

plt.imshow(distance[1][1])
plt.show()
plt.imshow(distance[2][1])
plt.show()

In [ ]:
for name, bin_im, coordinates in lakes:
    props = regionprops(bin_im)
    plt.imshow(bin_im)
    print(name)
    print(props[0]['orientation'])
    plt.show()
    

In [ ]:
with open('geojson/lacs_chablais.geojson') as f:
    data = json.load(f)

In [ ]:
x_coordinates, y_coordinates = extract_coordinates(data['features'][2])

## Binary image creation

In [ ]:
lac_des_rousses_bin = binary_image(im_size, x_coordinates, y_coordinates)

In [ ]:
plt.imshow(lac_des_rousses_bin)
plt.show()

### Unique Orientation

In [ ]:
def unique_orientation_coords(bin_image, x_coords, y_coords):
    props = regionprops(bin_image)

    angle = -props[0].orientation
    cos = np.cos(angle)
    sin = np.sin(angle)

    x_coords_rot = []
    y_coords_rot = []

    for i in range(len(x_coords)):
        x = x_coords[i]
        y = y_coords[i]

        x_coords_rot.append(x * cos - y * sin)
        y_coords_rot.append(y * cos + x * sin)
        
    return x_coords_rot, y_coords_rot

In [ ]:
plt.imshow(lac_des_rousses_bin)
plt.show()

x_coords_rot, y_coords_rot = unique_orientation_coords(lac_des_rousses_bin, x_coordinates, y_coordinates)
lac_des_rousses_bin_rot = binary_image(im_size, x_coords_rot, y_coords_rot)

plt.imshow(lac_des_rousses_bin_rot)
plt.show()

In [ ]:
for name, bin_image, coordinates in lakes:
    props = regionprops(bin_image)
    
    x_coords_rot, y_coords_rot = unique_orientation_coords(bin_image, coordinates[0], coordinates[1])
    bin_image_rot = binary_image(im_size, x_coords_rot, y_coords_rot)
    
    plt.imshow(bin_image_rot)
    plt.show()
    print(name)

### Skeleton vs Medial Axi vs Skeleton 3D

In [ ]:
skeleton = skeletonize(lac_des_rousses_bin)

In [ ]:
skeleton

In [ ]:
plt.imshow(skeleton)
plt.show()

In [ ]:
skel, distance = medial_axis(lac_des_rousses_bin, return_distance=True)

In [ ]:
plt.imshow(skel)
plt.show()

In [ ]:
plt.imshow(distance) #TODO use distance instead of 0-1 entries
plt.show()

In [ ]:
skel2, distance2 = medial_axis(np.ones((300, 300)), return_distance=True)

In [ ]:
np.max(distance2)

In [ ]:
from PIL import Image

In [ ]:
Image.fromarray(distance2).save('coucou.tiff')

In [ ]:
skeleton_3d = skeletonize_3d(lac_des_rousses_bin)

In [ ]:
plt.imshow(skeleton_3d)
plt.show()

In [ ]:
distances = []

for name, bin_image, coordinates in lakes:
    props = regionprops(bin_image)
    
    x_coords_rot, y_coords_rot = unique_orientation_coords(bin_image, coordinates[0], coordinates[1])
    bin_image_rot = binary_image(im_size, x_coords_rot, y_coords_rot)
    
    print(name)
    plt.imshow(bin_image_rot)
    plt.show()
    
    skel, distance = medial_axis(bin_image_rot, return_distance=True)
    
    distances.append({'name': name, 'distance': distance.tolist()})

In [ ]:
import json, os

In [ ]:
with open(os.path.join('vptree', 'images.dump'), 'w') as f:
    json.dump(distances, f)